##  Geek News 카드뉴스 생성기 (Google Colab 버전)

Google Colab 환경에서 실행할 수 있는 카드뉴스입니다
아래 단계에 따라 실행하면 카드뉴스를 생성할 수 있습니다.

### 1단계: 초기 설정 (최초 1회만 실행)

카드뉴스 생성에 필요한 라이브러리를 설치하고, 각종 설정 파일과 폴더를 생성합니다.

**[중요]** 아래 셀을 실행하기 전에, Colab 왼쪽의 **'파일' 아이콘**을 클릭하여 **세션 저장소**에 필요한 파일과 폴더를 먼저 업로드해야 합니다.

1.  **`image` 폴더 업로드**: 로컬의 `image` 폴더를 Colab의 `/content/` 경로에 업로드하세요. 캐릭터 이미지가 여기에 포함됩니다.
2.  **`QR.png` 파일 업로드**: 로컬의 `QR.png` 파일을 `/content/` 경로에 업로드하세요.
3.  **Hugging Face 토큰 설정**: `HUGGINGFACE_TOKEN`을 Colab의 **'비밀' 아이콘** (열쇠 모양)에 추가하는 것을 권장합니다. 또는, 아래 `%%writefile .env` 코드에 직접 토큰을 입력해도 됩니다.

In [ ]:
!pip install -q transformers torch python-dotenv httpx beautifulsoup4 lxml playwright nest_asyncio
!playwright install
import nest_asyncio
nest_asyncio.apply()

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

## 주의) 허깅 페이스 토큰 발급

* 허깅페이스 토큰을 미리 사이트에서 발급해 주세요 (무료)
* (1) https://huggingface.co/ 접속
* (2) 아이디가 없으신 분은 회원가입 진행
* (3) 이후 ACCESS_TOKEN 발급

```bash
HUGGINGFACE_TOKEN="<여기에 입력>"
```

In [ ]:
# 1-4: .env 파일 생성 (Hugging Face 토큰)
# [주의] 실제 토큰으로 바꿔주세요! (예: hf_xxxx...)
%%writefile .env
HUGGINGFACE_TOKEN=""

Overwriting .env


In [ ]:
# 1-5: 프로젝트 폴더 구조 생성
import os
os.makedirs('templates', exist_ok=True)
os.makedirs('output/images', exist_ok=True)

## html, CSS 스타일링

In [ ]:
# 1-7: HTML & CSS 템플릿 파일 생성
%%writefile templates/style.css
@import url('https://cdn.jsdelivr.net/gh/orioncactus/pretendard/dist/web/static/pretendard.css');
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+KR:wght@300;400;500;700&display=swap');
@import url('https://fonts.googleapis.com/css2?family=Black+Han+Sans&display=swap');

body {
    margin: 0;
    padding: 0;
    font-family: 'Pretendard', -apple-system, BlinkMacSystemFont, system-ui, Roboto, 'Helvetica Neue', 'Segoe UI', 'Apple SD Gothic Neo', 'Noto Sans KR', 'Malgun Gothic', 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol', sans-serif;
    background: #000;
    color: #333;
    overflow-x: hidden;
}

.page {
    width: {{page_width}}px;
    height: {{page_height}}px;
    display: flex;
    flex-direction: column;
    position: relative;
    page-break-after: always;
    box-sizing: border-box;
    overflow: hidden;
}

.card-container {
    width: 100%;
    height: 100%;
    padding: 60px;
    box-sizing: border-box;
    position: relative;
    display: flex;
    flex-direction: column;
}

/* 첫장 스타일 */
.cover {
    background: {{cover_background}};
}

.cover-container {
    position: relative;
    width: 100%;
    height: 100%;
    padding: 60px;
    box-sizing: border-box;
    display: flex;
    flex-direction: column;
    align-items: center;
    text-align: center;
}

.cover-header {
    margin-top: 120px;
}

.subtitle-small {
    font-size: 32px;
    color: rgba(255, 255, 255, 0.9);
    margin: 0;
    font-weight: 400;
    letter-spacing: 1px;
}

/* 뉴스 페이지 스타일 */
.news-page {
    background: {{news_background}};
}

/* 요약 페이지 스타일 */
.summary-page {
    background: {{summary_background}};
}

.summary-page .card-container {
    background: rgba(255, 255, 255, 0.12);
    border-radius: 30px;
    backdrop-filter: blur(22px);
    -webkit-backdrop-filter: blur(22px);
    border: 2px solid rgba(255, 255, 255, 0.35);
    box-shadow:
        0 20px 45px rgba(0,0,0,0.35),
        inset 0 0 45px rgba(255,255,255,0.12),
        inset 0 0 0 1px rgba(255,255,255,0.28);
}

.cover-main {
    flex: 1;
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
}

.title-main {
    font-size: {{cover_title_size}};
    color: rgba(255, 255, 255, 0.9);
    margin: 0 0 60px 0;
    font-weight: 900;
    text-shadow: 0 8px 30px rgba(0,0,0,0.3);
    letter-spacing: -3px;
}

.character-section {
    flex: 1;
    display: flex;
    justify-content: center;
    align-items: center;
    position: relative;
}

.main-character {
    max-width: 450px;
    max-height: 450px;
    object-fit: contain;
    filter: drop-shadow(0 10px 25px rgba(0,0,0,0.3));
}

.qr-section {
    position: absolute;
    top: 60px;
    right: 60px;
}

.qr-code {
    width: 150px;
    height: 150px;
    background: white;
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 5px 15px rgba(0,0,0,0.2);
}

.news-header-section {
    margin-bottom: 40px;
    text-align: center;
}

.topic-category {
    display: inline-block;
    background: white;
    color: {{cover_background}};
    padding: 8px 20px;
    border-radius: 20px;
    font-size: {{news_category_size}};
    font-weight: 600;
    margin-bottom: 20px;
}

.category-title {
    font-size: {{news_number_size}};
    color: white;
    text-align: center;
    margin: 0;
    font-weight: 700;
}

.news-content-section {
    flex: 1;
    display: flex;
    flex-direction: column;
    gap: 30px;
}

.news-body {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 20px;
    backdrop-filter: blur(18px);
    -webkit-backdrop-filter: blur(18px);
    padding: 50px;
    flex: 1;
}

.news-title {
    font-size: {{news_title_size}};
    font-weight: 800;
    color: rgba(255, 255, 255, 0.8);
    text-decoration: underline;
    text-decoration-color: {{cover_background}};
    text-decoration-thickness: 2px;
    text-decoration-style: solid;
    text-underline-offset: 4px;
    text-align: center;
    line-height: 1.3;
    margin: 0 0 20px 0;
    word-break: keep-all;
}

.news-description {
    font-size: {{news_description_size}};
    line-height: 1.6;
    color: #333;
    margin: 0;
    word-break: keep-all;
}

.links-section {
    margin-top: auto;
    background: rgba(255, 255, 255, 0.3);
    padding: 20px;
    border-radius: 15px;
}

.link-item {
    margin-bottom: 12px;
    font-size: {{link_text_size}};
    color: #333;
}

.link-label {
    font-weight: 700;
    margin-right: 10px;
    color: #333;
}

.page-character {
    position: absolute;
    bottom: 40px;
    right: 40px;
    max-width: 200px;
    max-height: 200px;
    object-fit: contain;
    filter: drop-shadow(0 8px 20px rgba(0,0,0,0.4));
}

.summary-header-section {
    text-align: center;
    margin-bottom: 50px;
}

.summary-main-title {
    font-size: {{summary_title_size}};
    color: white;
    margin: 0 0 20px 0;
    font-weight: 900;
    text-shadow: 0 6px 25px rgba(0,0,0,0.5);
}

.summary-date {
    font-size: 24px;
    color: rgba(255, 255, 255, 0.9);
    margin: 0;
    font-weight: 300;
    text-shadow: 0 2px 10px rgba(0,0,0,0.3);
}

.summary-content {
    flex: 1;
}

.summary-subtitle {
    font-size: {{summary_subtitle_size}};
    color: white;
    margin: 0 0 30px 0;
    font-weight: 700;
    text-align: center;
    text-shadow: 0 4px 15px rgba(0,0,0,0.4);
}

.summary-list {
    display: flex;
    flex-direction: column;
    gap: 15px;
    margin-bottom: 20px;
}

.summary-item {
    background: rgba(255, 255, 255, 0.18);
    padding: 25px;
    border-radius: 20px;
    backdrop-filter: blur(18px);
    -webkit-backdrop-filter: blur(18px);
    border: 1px solid rgba(255, 255, 255, 0.3);
    box-shadow:
        0 10px 25px rgba(0,0,0,0.3),
        inset 0 0 20px rgba(255,255,255,0.1),
        inset 0 0 0 1px rgba(255,255,255,0.25);
}

.summary-header {
    display: flex;
    align-items: center;
    gap: 15px;
    margin-bottom: 15px;
}

.summary-number {
    background: rgba(255, 255, 255, 0.9);
    color: #333;
    width: 40px;
    height: 40px;
    border-radius: 50%;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: 700;
    font-size: 18px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.2);
}

.summary-category {
    background: rgba(255, 255, 255, 0.25);
    color: white;
    padding: 6px 15px;
    border-radius: 15px;
    font-size: 14px;
    font-weight: 600;
    backdrop-filter: blur(10px);
    -webkit-backdrop-filter: blur(10px);
}

.summary-title {
    font-size: {{summary_item_title_size}};
    color: white;
    margin: 0;
    font-weight: 600;
    line-height: 1.4;
    text-shadow: 0 2px 10px rgba(0,0,0,0.4);
}

.summary-footer {
    text-align: center;
    padding-top: 30px;
    border-top: 2px solid rgba(255, 255, 255, 0.3);
}

.summary-footer p {
    color: rgba(255, 255, 255, 0.95);
    font-size: 20px;
    margin: 10px 0;
    font-weight: 500;
    text-shadow: 0 2px 8px rgba(0,0,0,0.3);
}

.summary-source {
    font-size: 18px !important;
    color: rgba(255, 255, 255, 0.8) !important;
    font-weight: 300 !important;
}

@media print {
    body {
        margin: 0;
        background: white;
    }
    .page {
        page-break-after: always;
        margin: 0;
        width: {{page_width}}px;
        height: {{page_height}}px;
    }
}

Overwriting templates/style.css


In [ ]:
%%writefile templates/cover_template.html
<div class="page cover">
    <div class="cover-container">
        <div class="cover-header">
            <p class="subtitle-small">{cover_subtitle}</p>
        </div>
        <div class="cover-main">
            <div class="title-main">{cover_title}</div>
            <div class="character-section">
                {character_image}
            </div>
        </div>
        {qr_section}
    </div>
</div>

Overwriting templates/cover_template.html


In [ ]:
%%writefile templates/news_template.html
<div class="page news-page">
    <div class="card-container">
        <div class="news-header-section">
            <div class="topic-category">{topic_category}</div>
            <h2 class="category-title">GeekNews #{page_number}</h2>
        </div>
        <div class="news-content-section">
            <div class="news-title">{news_title}</div>
            <div class="news-body">
                <div class="news-description">{news_description}</div>
            </div>
            <div class="links-section">
                {links}
            </div>
        </div>
        {character_image}
    </div>
</div>

Overwriting templates/news_template.html


In [ ]:
%%writefile templates/summary_template.html
<div class="page summary-page">
    <div class="card-container">
        <div class="summary-header-section">
            <h1 class="summary-main-title">{summary_title}</h1>
            <p class="summary-date">{summary_date}</p>
        </div>
        <div class="summary-content">
            <h2 class="summary-subtitle">{summary_subtitle}</h2>
            <div class="summary-list">
                {summary_items}
            </div>
            <div class="summary-footer">
                <p>{summary_footer}</p>
                <p class="summary-source">{summary_source}</p>
            </div>
        </div>
    </div>
</div>

Overwriting templates/summary_template.html


In [ ]:
%%writefile templates/summary_item_template.html
<div class="summary-item">
    <div class="summary-header">
        <span class="summary-number">{number}</span>
        <span class="summary-category">{category}</span>
    </div>
    <div class="summary-title">{title}</div>
</div>

Overwriting templates/summary_item_template.html


In [ ]:
%%writefile templates/combined_template.html
<html>
<head>
    <meta charset="UTF-8">
    <title>오늘의 긱뉴스</title>
    <style>
        {{ base_css }}
        body {
            margin: 0;
            padding: 0;
        }
        #wrapper {
            width: {{ page_width }}px;
            background: {{ cover_background }};
        }
        .page {
            height: {{ page_height }}px;
            page-break-after: unset;
            background: transparent !important;
        }
        .card-container, .cover-container, .summary-page .card-container {
            background: transparent !important;
            box-shadow: none !important;
            border: none !important;
        }
    </style>
</head>
<body>
    <div id="wrapper">
        {{ html_content }}
    </div>
</body>
</html>

Overwriting templates/combined_template.html


## 수정 가능 (1) - CONFIG

In [ ]:
# 1-6: config.py 파일 생성
%%writefile config.py
# 파일 경로 설정
PATH_CONFIG = {
    "output_dir": "output",
    "template_dir": "templates",
    "style_file": "style.css",
    "cover_template": "cover_template.html",
    "news_template": "news_template.html",
    "summary_template": "summary_template.html",
    "summary_item_template": "summary_item_template.html",
    "summary_prompt": "summary_prompt.txt",
    "image_dir": "output/images",
    "character_dir": "image/character"
}

# 크롤링 설정
CRAWLING_CONFIG = {
    "news_count": 5,  # 가져올 뉴스 개수
    "base_url": "https://news.hada.io/",
    "timeout": 30  # HTTP 요청 타임아웃 (초)
}

# PDF/이미지 생성 설정
OUTPUT_CONFIG = {
    "page_width": 1920,
    "page_height": 1080,
    "pdf_margin": {'top': '0px', 'right': '0px', 'bottom': '0px', 'left': '0px'},
    "image_quality": 95,  # JPG 품질 (1-100)
    "generate_png": True,
    "generate_jpg": True,
    "generate_pdf": True
}

# 이미지 설정
IMAGE_CONFIG = {
    "main_character": "1",
    "all_characters": "all",
    "character_names": ["1", "2", "3", "4", "5", "6", "7"]
}

# S3 설정
S3_CONFIG = {
    "use_s3": True,
    "bucket_name": "jiggloghttps",
    "region": "ap-northeast-2",
    "base_url": "https://jiggloghttps.s3.ap-northeast-2.amazonaws.com/",
    "character_prefix": "image/",
    "qr_code_key": "image/QR.png",
    "font_prefix": "fonts/"
}

# 색상 커스터마이징 설정
COLOR_CONFIG = {
    "cover_background": "linear-gradient(160deg, #FF5F6D 0%, #FFC371 100%)",
    "news_background": "linear-gradient(160deg, #FF5F6D 0%, #FFC371 100%)",
    "summary_background": "linear-gradient(160deg, #FF5F6D 0%, #FFC371 100%)",
    "end_background": "linear-gradient(160deg, #FF5F6D 0%, #FFC371 100%)"
}

# 폰트 크기 커스터마이징 설정
FONT_CONFIG = {
    "cover_title": "200px",
    "cover_subtitle": "50px",
    "news_title": "48px",
    "news_description": "32px",
    "news_category": "18px",
    "news_number": "36px",
    "link_text": "18px",
    "summary_title": "72px",
    "summary_subtitle": "36px",
    "summary_item_title": "22px"
}

# 텍스트 커스터마이징 설정
TEXT_CONFIG = {
    "cover_subtitle": "모여봐요 개발자와 AI의 숲",
    "cover_title": "모드뉴스",
    "summary_title": "GeekNews 요약",
    "summary_subtitle": "오늘의 주요 뉴스",
    "summary_footer_text": "총 {count}개의 뉴스를 확인했어요",
    "summary_source": "출처: GeekNews (news.hada.io)"
}
# AI 모델 설정
AI_CONFIG = {
    "model_name": "lcw99/t5-large-korean-text-summary",
    "max_input_length": 768,
    "max_output_length": 150,
    "min_output_length": 50,
    "length_penalty": 2.0,
    "num_beams": 4
}

Overwriting config.py


## 수정 가능 (2) - 프롬프트

In [ ]:
%%writefile templates/summary_prompt.txt
다음 내용을 완전한 문장으로 요약하되, 부드럽고 친근한 문체로 작성해 주세요.

요약할 내용: {text}

요약은 모든 내용을 포함하게 기승전결로 작성해주세요

Overwriting templates/summary_prompt.txt


## 수정 가능 (3) - 코드

In [ ]:
import os
import asyncio
from datetime import datetime
from bs4 import BeautifulSoup
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from config import *
from dotenv import load_dotenv
import httpx
from playwright.async_api import async_playwright

load_dotenv()

class GeekNewsCardGenerator:
    def __init__(self):
        self.output_dir = PATH_CONFIG["output_dir"]
        self.template_dir = PATH_CONFIG["template_dir"]

        print("한국어 요약 모델 로딩 중...")
        hf_token = os.getenv('HUGGINGFACE_TOKEN')
        self.tokenizer = AutoTokenizer.from_pretrained(
            AI_CONFIG["model_name"], token=hf_token, trust_remote_code=True
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            AI_CONFIG["model_name"], token=hf_token
        )
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        print("모델 로딩 완료")

    def summarize_text(self, text):
        if not text or len(text.strip()) < 50:
            print(f"  [요약 건너뜀] 텍스트가 너무 짧습니다: {text[:100]}")
            return ""
        print(f"  [요약 원문] {text[:150]}...")
        try:
            prompt_template = self.load_template(PATH_CONFIG["summary_prompt"])
            prompt = prompt_template.format(text=text)
            inputs = self.tokenizer(
                prompt,
                max_length=AI_CONFIG["max_input_length"],
                truncation=True,
                return_tensors="pt"
            ).to(self.device)
            summary_ids = self.model.generate(
                inputs["input_ids"],
                max_length=AI_CONFIG["max_output_length"],
                min_length=AI_CONFIG["min_output_length"],
                length_penalty=AI_CONFIG["length_penalty"],
                num_beams=AI_CONFIG["num_beams"],
                early_stopping=True
            )
            summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            print(f"  [요약 성공] {summary.strip()}")
            return summary.strip()
        except Exception as e:
            print(f"  [요약 실패] 오류: {e}")
            return "요약 생성에 실패했습니다."

    async def get_detail(self, topic_id):
        try:
            async with httpx.AsyncClient() as client:
                url = f'https://news.hada.io/topic?id={topic_id}'
                response = await client.get(url)
                soup = BeautifulSoup(response.text, 'lxml')
                contents_elem = soup.find('div', class_='topic_contents')
                if contents_elem:
                    return contents_elem.get_text(separator=' ', strip=True)
                desc_elem = soup.find('div', class_='topic_desc')
                if desc_elem:
                    return desc_elem.get_text(strip=True)
                return ""
        except Exception as e:
            print(f"긱뉴스 상세 정보 가져오기 실패: {e}")
            return ""

    async def fetch_news(self):
        async with httpx.AsyncClient(timeout=CRAWLING_CONFIG["timeout"]) as client:
            response = await client.get(CRAWLING_CONFIG["base_url"])
            soup = BeautifulSoup(response.text, 'lxml')
            news_items = []
            topics = soup.find_all('div', class_='topic_row')[:CRAWLING_CONFIG["news_count"]]
            for topic in topics:
                title_elem = topic.find('div', class_='topictitle')
                if not title_elem: continue
                title_link = title_elem.find('a')
                if not title_link: continue

                title = title_link.text.strip()
                original_link = title_link.get('href', '')

                topic_id = None
                all_links = topic.find_all('a')
                for link in all_links:
                    href = link.get('href', '')
                    if 'topic?id=' in href:
                        try:
                            topic_id = href.split('id=')[-1].split('&')[0]
                            break
                        except:
                            pass

                desc_elem = topic.find('span', class_='topicdesc')
                desc = desc_elem.text.strip() if desc_elem else ''

                if topic_id:
                    detailed_desc = await self.get_detail(topic_id)
                    if detailed_desc and len(detailed_desc) > len(desc):
                        desc = detailed_desc

                summarized_desc = self.summarize_text(desc)
                news_items.append({
                    'title': title,
                    'description': summarized_desc if summarized_desc else "요약 정보가 없습니다.",
                    'link': original_link,
                    'topic_id': topic_id
                })
            return news_items

    def get_character_sources(self):
        """S3 URL로부터 캐릭터 이미지 소스를 생성합니다."""
        characters = {}
        base_url = S3_CONFIG["base_url"]
        prefix = S3_CONFIG["character_prefix"]
        for name in IMAGE_CONFIG["character_names"]:
            characters[name] = f"{base_url}{prefix}{name}.png"
        return characters

    def load_template(self, template_name):
        template_path = os.path.join(self.template_dir, template_name)
        return self.read_file(template_path)

    def _render_cover_page(self, character_src, qr_src):
        """커버 페이지 HTML을 렌더링합니다."""
        cover_template = self.load_template(PATH_CONFIG["cover_template"])
        character_html = f'<img src="{character_src}" class="character main-character" alt="캐릭터" />' if character_src else ''
        qr_html = f'<div class="qr-section"><img src="{qr_src}" class="qr-code" alt="QR코드" /></div>' if qr_src else ''

        return cover_template.format(
            cover_subtitle=TEXT_CONFIG["cover_subtitle"],
            cover_title=TEXT_CONFIG["cover_title"],
            character_image=character_html,
            qr_section=qr_html
        )

    def _render_news_pages(self, news_items, page_characters):
        """뉴스 페이지들의 HTML을 렌더링합니다."""
        news_template = self.load_template(PATH_CONFIG["news_template"])
        pages_html = ""

        for page_index, news in enumerate(news_items):
            character_src = page_characters[page_index % len(page_characters)] if page_characters else None

            domain_label = "원문"
            if news.get('link'):
                if "github.com" in news['link']: domain_label = "GitHub"
                elif "youtube.com" in news['link'] or "youtu.be" in news['link']: domain_label = "YouTube"
                elif "blog" in news['link'] or "medium.com" in news['link']: domain_label = "블로그"
                elif "news" in news['link']: domain_label = "뉴스"

            geek_news_link = f"https://news.hada.io/topic?id={news['topic_id']}" if news.get('topic_id') else ""

            topic_category = "기술"
            link_lower = news.get('link', '').lower()
            title_lower = news.get('title', '').lower()
            if "github" in link_lower or "git" in title_lower: topic_category = "개발"
            elif "youtube" in link_lower: topic_category = "영상"
            elif "blog" in link_lower: topic_category = "블로그"
            elif "ai" in title_lower or "gemini" in title_lower: topic_category = "AI"
            elif "데이터" in title_lower or "data" in title_lower: topic_category = "데이터"

            links_html = ""
            if geek_news_link:
                links_html += f'<div class="link-item"><span class="link-label">토론:</span>{geek_news_link}</div>'
            if news.get('link'):
                links_html += f'<div class="link-item"><span class="link-label">{domain_label}:</span>{news["link"]}</div>'
            character_html = f'<img src="{character_src}" class="page-character" alt="캐릭터" />' if character_src else ''

            pages_html += news_template.format(
                topic_category=topic_category,
                page_number=page_index + 1,
                news_title=news['title'],
                news_description=news['description'],
                links=links_html,
                character_image=character_html
            )
        return pages_html

    def _render_summary_page(self, news_items):
        """요약 페이지 HTML을 렌더링합니다."""
        current_date = datetime.now().strftime("%Y년 %m월 %d일")
        summary_item_template = self.load_template(PATH_CONFIG["summary_item_template"])
        summary_items_html = ""

        for index, news in enumerate(news_items, 1):
            topic_category = "기술"
            link_lower = news.get('link', '').lower()
            title_lower = news.get('title', '').lower()
            if "github" in link_lower or "git" in title_lower: topic_category = "개발"
            elif "ai" in title_lower or "gemini" in title_lower: topic_category = "AI"
            elif "데이터" in title_lower or "data" in title_lower: topic_category = "데이터"

            summary_items_html += summary_item_template.format(
                number=index,
                category=topic_category,
                title=news['title']
            )

        summary_template = self.load_template(PATH_CONFIG["summary_template"])
        return summary_template.format(
            summary_title=TEXT_CONFIG["summary_title"],
            summary_date=current_date,
            summary_subtitle=TEXT_CONFIG["summary_subtitle"],
            summary_items=summary_items_html,
            summary_footer=TEXT_CONFIG["summary_footer_text"].format(count=len(news_items)),
            summary_source=TEXT_CONFIG["summary_source"]
        )

    async def create_html(self, news_items):
        """모든 페이지의 HTML을 생성하고 결합합니다."""
        available_characters = self.get_character_sources()
        main_character_src = available_characters.get(IMAGE_CONFIG["main_character"])
        page_characters = [
            src for name, src in available_characters.items()
            if name != IMAGE_CONFIG["main_character"] and name != IMAGE_CONFIG["all_characters"]
        ]
        qr_src = f"{S3_CONFIG['base_url']}{S3_CONFIG['qr_code_key']}"
        cover_html = self._render_cover_page(main_character_src, qr_src)
        news_html = self._render_news_pages(news_items, page_characters)
        summary_html = self._render_summary_page(news_items)

        return f"{cover_html}{news_html}{summary_html}"

    def create_styles(self):
        """템플릿과 설정값을 결합하여 최종 CSS를 생성합니다."""
        base_css = self.load_template(PATH_CONFIG["style_file"])

        replacements = {
            "{{page_width}}": str(OUTPUT_CONFIG["page_width"]),
            "{{page_height}}": str(OUTPUT_CONFIG["page_height"]),
            "{{cover_background}}": COLOR_CONFIG["cover_background"],
            "{{news_background}}": COLOR_CONFIG["news_background"],
            "{{summary_background}}": COLOR_CONFIG["summary_background"],
            "{{end_background}}": COLOR_CONFIG["end_background"],
            "{{cover_title_size}}": FONT_CONFIG["cover_title"],
            "{{cover_subtitle_size}}": FONT_CONFIG["cover_subtitle"],
            "{{news_title_size}}": FONT_CONFIG["news_title"],
            "{{news_description_size}}": FONT_CONFIG["news_description"],
            "{{news_category_size}}": FONT_CONFIG["news_category"],
            "{{news_number_size}}": FONT_CONFIG["news_number"],
            "{{link_text_size}}": FONT_CONFIG["link_text"],
            "{{summary_title_size}}": FONT_CONFIG["summary_title"],
            "{{summary_subtitle_size}}": FONT_CONFIG["summary_subtitle"],
            "{{summary_item_title_size}}": FONT_CONFIG["summary_item_title"],
        }

        customized_css = base_css
        for placeholder, value in replacements.items():
            customized_css = customized_css.replace(placeholder, value)

        return customized_css

    async def generate_all(self, html_content, css_content):
        """개별 페이지 PDF와 이미지들을 생성합니다."""
        final_html = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <title>오늘의 긱뉴스</title>
            <style>{css_content}</style>
        </head>
        <body>{html_content}</body>
        </html>
        """
        timestamp = datetime.now().strftime("%Y%m%d%H%M")
        pdf_filename = f"geek_news_{timestamp}.pdf"
        html_output_path = os.path.join(self.output_dir, "geek_news.html")
        pdf_output_path = os.path.join(self.output_dir, pdf_filename)

        with open(html_output_path, "w", encoding="utf-8") as f:
            f.write(final_html)
        print(f"'{html_output_path}' 파일 생성 완료")

        html_path_url = f'file://{os.path.abspath(html_output_path)}'

        async with async_playwright() as p:
            browser = await p.chromium.launch()
            page = await browser.new_page()
            await page.set_viewport_size({"width": OUTPUT_CONFIG["page_width"], "height": OUTPUT_CONFIG["page_height"]})
            await page.goto(html_path_url, wait_until='networkidle')
            await page.wait_for_timeout(2000)

            if OUTPUT_CONFIG["generate_pdf"]:
                await page.pdf(
                    path=pdf_output_path,
                    width=f"{OUTPUT_CONFIG['page_width']}px",
                    height=f"{OUTPUT_CONFIG['page_height']}px",
                    print_background=True,
                    margin=OUTPUT_CONFIG["pdf_margin"]
                )
                print(f"'{pdf_output_path}' 파일 생성 완료")

            pages = await page.query_selector_all('.page')
            print(f"총 {len(pages)} 페이지 이미지 생성 중...")
            image_dir = PATH_CONFIG["image_dir"]
            for i, page_element in enumerate(pages, 1):
                if OUTPUT_CONFIG["generate_png"]:
                    png_filename = f"geek_page_{i:02d}.png"
                    png_output_path = os.path.join(image_dir, png_filename)
                    await page_element.screenshot(path=png_output_path, type='png', omit_background=False)
                    print(f"'{png_filename}' 생성 완료")

                if OUTPUT_CONFIG["generate_jpg"]:
                    jpg_filename = f"geek_page_{i:02d}.jpg"
                    jpg_output_path = os.path.join(image_dir, jpg_filename)
                    await page_element.screenshot(path=jpg_output_path, type='jpeg', quality=OUTPUT_CONFIG["image_quality"], omit_background=False)
                    print(f"'{jpg_filename}' 생성 완료")

            await browser.close()

    async def generate_combined_image(self, news_items):
        """모든 페이지를 합친 단일 이미지를 생성합니다."""
        print("연속 그라데이션을 적용하여 이미지를 결합하는 중...")

        template_str = self.load_template("combined_template.html")
        if not template_str:
            print("통합 이미지 템플릿을 찾을 수 없습니다.")
            return

        html_content = await self.create_html(news_items)
        base_css = self.create_styles()

        final_html = template_str.replace("{{ base_css }}", base_css)
        final_html = final_html.replace("{{ html_content }}", html_content)
        final_html = final_html.replace("{{ page_width }}", str(OUTPUT_CONFIG["page_width"]))
        final_html = final_html.replace("{{ page_height }}", str(OUTPUT_CONFIG["page_height"]))
        final_html = final_html.replace("{{ cover_background }}", COLOR_CONFIG["cover_background"])

        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        combined_html_path = os.path.join(self.output_dir, f"combined_view_{timestamp}.html")
        combined_image_path = os.path.join(self.output_dir, "geek_news.png")

        with open(combined_html_path, "w", encoding="utf-8") as f:
            f.write(final_html)

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                html_path_url = f'file://{os.path.abspath(combined_html_path)}'

                await page.goto(html_path_url, wait_until='networkidle')
                await page.locator('#wrapper').screenshot(path=combined_image_path)
                await browser.close()

            print(f"'{combined_image_path}' 파일 생성 완료")

        except Exception as e:
            print(f"통합 이미지 생성 실패: {e}")

        finally:
            if os.path.exists(combined_html_path):
                os.remove(combined_html_path)

    def read_file(self, filepath):
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                return f.read()
        except FileNotFoundError:
            print(f"오류: 파일 '{filepath}'을(를) 찾을 수 없습니다.")
            return ""

    def create_directory(self):
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(PATH_CONFIG["image_dir"], exist_ok=True)

    async def generate(self):
        print("=== 긱뉴스 카드뉴스 생성 시작 ===")
        self.create_directory()
        news_items = await self.fetch_news()
        if not news_items:
            print("뉴스를 가져올 수 없습니다.")
            return
        print(f"{len(news_items)}개의 뉴스를 가져왔습니다.")
        html_content = await self.create_html(news_items)
        css_content = self.create_styles()
        await self.generate_all(html_content, css_content)
        await self.generate_combined_image(news_items)
        print("=== 긱뉴스 카드뉴스 생성 완료 ===")

async def main():
    generator = GeekNewsCardGenerator()
    await generator.generate()

# 실행
await main()

한국어 요약 모델 로딩 중...


OSError: There was a specific connection error when trying to load lcw99/t5-large-korean-text-summary:
401 Client Error: Unauthorized for url: https://huggingface.co/lcw99/t5-large-korean-text-summary/resolve/main/config.json (Request ID: Root=1-685f7c0b-21dfe797746965827226fc60;f1b53f59-8657-4627-9b64-352e0be82c34)

Invalid credentials in Authorization header